In [1]:

import pandas as pd

import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
import datetime

import speech_recognition as sr
import pyttsx3 
import pyaudio
import pickle

In [2]:
#data

data = pd.read_csv("Chatbot Data.csv", encoding = "ISO-8859-1", engine='python')

#data.drop(['Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace=True)

data.columns = ['Question','Answer','Class']

questions = data['Question'].values



In [3]:
stop_words = set(stopwords.words('english'))

def cleanup(sentence):
    
    word_tok = nltk.word_tokenize(sentence)
    stemmed_words = [w for w in word_tok if not w in stop_words]
    return ' '.join(stemmed_words)
    

In [4]:
X = []

for question in questions:
    X.append(cleanup(str(question)))

In [5]:
#loading model

with open('model_pkl' , 'rb') as f:
    model = pickle.load(f)

In [6]:
#defining entities

le = LE()

from sentence_transformers import SentenceTransformer

t_model = SentenceTransformer("Tranformer_saved")

le.fit(data['Class'])

le.transform(data['Class'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [7]:
def get_response(usrText):
    
    t_usr = t_model.encode([cleanup(usrText.strip().lower())])
    class_ = le.inverse_transform(model.predict(t_usr))


    questionset = data[data['Class'].values == class_]

    cos_sims = []
    for question in questionset['Question']:
        sims = cosine_similarity(t_model.encode([question]), t_usr)

        cos_sims.append(sims)

    ind = cos_sims.index(max(cos_sims))

    b = [questionset.index[ind]]
    
    r = data['Answer'][questionset.index[ind]]+"   "
    
    return r


In [8]:
from flask import Flask, render_template, request

app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return get_response(userText)



if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Mar/2022 21:37:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 21:37:05] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 21:37:06] "GET /Icons/mic.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [30/Mar/2022 21:37:12] "GET /Icons/person.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [30/Mar/2022 21:37:12] "GET /get?msg=google.com HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 21:37:12] "GET /Icons/bot.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [30/Mar/2022 21:37:19] "GET /Icons/person.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [30/Mar/2022 21:37:19] "GET /get?msg=NEFT HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 21:37:19] "GET /Icons/bot.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [30/Mar/2022 21:37:24] "GET /Icons/person.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [30/Mar/2022 21:37:24] "GET /get?msg=RTGS HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 21:37:24] "GET /Icons/bot.jpeg HTTP/1.1" 404 -
